# 用于预训练BERT的数据集
:label:`sec_bert-dataset`

为了预训练 :numref:`sec_bert`中实现的BERT模型，我们需要以理想的格式生成数据集，以便于两个预训练任务：遮蔽语言模型和下一句预测。一方面，最初的BERT模型是在两个庞大的图书语料库和英语维基百科（参见 :numref:`subsec_bert_pretraining_tasks`）的合集上预训练的，但它很难吸引这本书的大多数读者。另一方面，现成的预训练BERT模型可能不适合医学等特定领域的应用。因此，在定制的数据集上对BERT进行预训练变得越来越流行。为了方便BERT预训练的演示，我们使用了较小的语料库WikiText-2 :cite:`Merity.Xiong.Bradbury.ea.2016`。

与 :numref:`sec_word2vec_data`中用于预训练word2vec的PTB数据集相比，WikiText-2（1）保留了原来的标点符号，适合于下一句预测；（2）保留了原来的大小写和数字；（3）大了一倍以上。


In [89]:
import os
import random
import torch
from d2l import torch as d2l
from datasets import load_dataset

在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。保留至少有两句话的段落。为了简单起见，我们仅使用句号作为分隔符来拆分句子。我们将更复杂的句子拆分技术的讨论留在本节末尾的练习中。


In [90]:
def load_wikitext_2():
    # 从 Hugging Face 下载 wikitext-2 数据集
    # 数据集大小：12.1 MB
    wikitext2 = load_dataset("wikitext", name="wikitext-2-v1", cache_dir="D:\Github\AI\d2l-pytorch\data")
    # 拆分数据集
    train_iter = wikitext2["train"]
    valid_iter = wikitext2["validation"]
    test_iter = wikitext2["test"]
    return train_iter, valid_iter, test_iter


In [91]:
def _read_wiki():
    train_iter, _, _ = load_wikitext_2()
    # 将数据集转换为列表
    # lines:List[Str]
    lines = [line['text'] for line in train_iter]
    # paragraphs:List[List[Str]]
    # 只有当你对字符串主动调用会返回列表的方法时，才会产生子列表,比如这里的split()
    paragraphs = [line.strip().lower().split(' . ') for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs
    # paragraphs:List[List[Str]]


In [92]:
paragraphs = _read_wiki()
# for 行内所有句子 in 行
for paragraph in paragraphs[:10]:
    print(paragraph)


['jacob anthony degrom ( born june 19 , 1988 ) , is an american professional baseball pitcher for the new york mets of major league baseball ( mlb )', 'prior to playing professionally , degrom attended stetson university and played college baseball for the stetson hatters .']
['the notion that vikings used a. muscaria to produce their <unk> <unk> was first suggested by the swedish professor samuel <unk> in 1784', '<unk> based his theories on reports about the use of fly agaric among siberian shamans', 'the notion has become widespread since the 19th century , but no contemporary sources mention this use or anything similar in their description of <unk>', 'muscimol is generally a mild <unk> , but it can create a range of different reactions within a group of people', 'it is possible that it could make a person angry , or cause them to be " very <unk> or sad , jump about , dance , sing or give way to great <unk> " .']
["the day after the young turks ' press conference , taylor privately 

## 为预训练任务定义辅助函数

在下文中，我们首先为BERT的两个预训练任务实现辅助函数。这些辅助函数将在稍后将原始文本语料库转换为理想格式的数据集时调用，以预训练BERT。

### 生成下一句预测任务的数据

根据 :numref:`subsec_nsp`的描述，`_get_next_sentence`函数生成二分类任务的训练样本。


In [93]:
# sentence: 当前句子
# next_sentence: 下一个句子
# paragraphs: 所有行
def _get_next_sentence(sentence, next_sentence, paragraphs):
    # 抛硬币决定是否是下一个句子
    if random.random() < 0.5:
        is_next = True
    else:
        # 替换掉下一句
        # 随机选择一行，然后随机选择一个句子
        # 有可能选到当前句子或者当前句子的下一句，这个算噪声
        # paragraph:List[List[Str]]
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

下面的函数通过调用`_get_next_sentence`函数从输入`paragraph`生成用于下一句预测的训练样本。这里`paragraph`是句子列表，其中每个句子都是词元列表。自变量`max_len`指定预训练期间的BERT输入序列的最大长度。


In [ ]:
#@save
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    # 遍历1行除了最后一个句子外的所有句子
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        # 拼接tokens_a和tokens_b，添加特殊词元
        # tokens:['<cls>', 'hello', 'world', '<sep>', 'how', 'are', 'you', '<sep>']
        # segments:[0, 0, 0, 0, 1, 1, 1, 1]
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        # nsp_data_from_paragraph:List[Tuple[List[str], List[int], bool]]:每个元素是一个元组(tokens, segments, is_next)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

### 生成遮蔽语言模型任务的数据
:label:`subsec_prepare_mlm_data`

为了从BERT输入序列生成遮蔽语言模型的训练样本，我们定义了以下`_replace_mlm_tokens`函数。在其输入中，`tokens`是表示BERT输入序列的词元的列表，`candidate_pred_positions`是不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测），以及`num_mlm_preds`指示预测的数量（选择15%要预测的随机词元）。在 :numref:`subsec_mlm`中定义遮蔽语言模型任务之后，在每个预测位置，输入可以由特殊的“掩码”词元或随机词元替换，或者保持不变。最后，该函数返回可能替换后的输入词元、发生预测的词元索引和这些预测的标签。


In [95]:
# tokens  --mask-->  mlm_input_tokens
# 这段代码实现的是 BERT 模型中 Masked Language Model (MLM) 任务的核心逻辑
# 对输入序列中的某些词元（tokens）进行遮蔽（masking）或替换，同时记录被遮蔽位置及其原始标签，用于后续的 MLM 训练任务。
# tokens:List[str]:['<cls>', 'hello', 'world', '<sep>', 'how', 'are', 'you', '<sep>']
# candidate_pred_positions: 候选预测位置的索引列表(排除掉'<cls>'和'<sep>'的位置索引)
# num_mlm_preds: 要预测的遮蔽词元数量 = tokens长度 * 0.15 四舍五入到最近的整数
# vocab: 词汇表对象，用于将索引转换为词元
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    # 深拷贝副本，为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens] # 深拷贝
    # 记录被遮蔽位置及其原始标签
    pred_positions_and_labels = [] 
    # 打乱可遮蔽位置(候选位置)的索引列表
    random.shuffle(candidate_pred_positions)

    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
    
        # 这正是 BERT 论文中描述的 MLM 的 80-10-10 策略
        if random.random() < 0.8:
            masked_token = '<mask>' # 80%的时间：将词替换为“<mask>”词元
        else:
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position] # 10%的时间：保持词不变
            else:
                masked_token = random.choice(vocab.idx_to_token) # 10%的时间：用随机词替换该词

        # 实施词元替换
        mlm_input_tokens[mlm_pred_position] = masked_token
        # 记录被遮蔽位置及其原始标签(真实值)
        pred_positions_and_labels.append((mlm_pred_position, tokens[mlm_pred_position]))

    return mlm_input_tokens, pred_positions_and_labels
    # mlm_input_tokens: 更新后的tokens,包含遮蔽语言模型任务输入的词元列表
    # pred_positions_and_labels: 被遮蔽位置及其原始标签的元组列表

通过调用前述的`_replace_mlm_tokens`函数，以下函数将BERT输入序列（`tokens`）作为输入，并返回输入词元的索引（在 :numref:`subsec_mlm`中描述的可能的词元替换之后）、发生预测的词元索引以及这些预测的标签索引。


In [96]:
# tokens:List[str]:['<cls>', 'hello', 'world', '<sep>', 'how', 'are', 'you', '<sep>']
# 这个函数只能处理单句的tokens, 不能处理批量的tokens
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        # 排除'<cls>'和'<sep>'的位置索引
        candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15)) # round：四舍五入到最近的整数
    # tokens  --mask-->  mlm_input_tokens
    # pred_positions_and_labels: (position, label)
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab)
    # 对于列表中的每一个元素 x，取它的第 0 个元素（即 x[0]）作为排序的“关键字”。
    # 重新回到原始的tokens顺序
    pred_positions_and_labels = sorted(pred_positions_and_labels, key=lambda x: x[0])
    # 位置索引
    pred_positions = [v[0] for v in pred_positions_and_labels]
    # 预测标签
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    # vacab[mlm_input_tokens]: 将mlm_input_tokens(添加mask后的tokens)中的每个词元转换为对应的索引
    # vacab[mlm_pred_labels]: 将mlm_pred_labels(被遮蔽位置的原始标签)中的每个词元转换为对应的索引
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]
    # (input_ids, pred_positions, pred_labels)

## 将文本转换为预训练数据集

现在我们几乎准备好为BERT预训练定制一个`Dataset`类。在此之前，我们仍然需要定义辅助函数`_pad_bert_inputs`来将特殊的“&lt;mask&gt;”词元附加到输入。它的参数`examples`包含来自两个预训练任务的辅助函数`_get_nsp_data_from_paragraph`和`_get_mlm_data_from_tokens`的输出。


In [97]:
# examples:(mlm_input_ids, pred_positions, mlm_pred_labels_ids, segments, is_next)
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15) # 四舍五入
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels_ids = [], [], []
    nsp_labels = []

    for (mlm_input_ids, pred_positions, mlm_pred_label_ids, segments, is_next) in examples:
        # 填充词元到最大长度
        all_token_ids.append(torch.tensor(mlm_input_ids + [vocab['<pad>']] * (max_len - len(mlm_input_ids)), dtype=torch.long))
        # 填充句子对的片段索引到最大长度
        all_segments.append(torch.tensor(segments + [0] * (max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(mlm_input_ids), dtype=torch.float32))
        # 填充预测位置到最大预测数
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.float32))
        all_mlm_labels_ids.append(torch.tensor(mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (
    all_token_ids,           # [batch_size, max_len]
    all_segments,            # [batch_size, max_len]
    valid_lens,              # [batch_size]
    all_pred_positions,      # [batch_size, max_num_mlm_preds]
    all_mlm_weights,         # [batch_size, max_num_mlm_preds]
    all_mlm_labels_ids,      # [batch_size, max_num_mlm_preds]
    nsp_labels               # [batch_size]
)

将用于生成两个预训练任务的训练样本的辅助函数和用于填充输入的辅助函数放在一起，我们定义以下`_WikiTextDataset`类为用于预训练BERT的WikiText-2数据集。通过实现`__getitem__ `函数，我们可以任意访问WikiText-2语料库的一对句子生成的预训练样本（遮蔽语言模型和下一句预测）样本。

最初的BERT模型使用词表大小为30000的WordPiece嵌入 :cite:`Wu.Schuster.Chen.ea.2016`。WordPiece的词元化方法是对 :numref:`subsec_Byte_Pair_Encoding`中原有的字节对编码算法稍作修改。为简单起见，我们使用`d2l.tokenize`函数进行词元化。出现次数少于5次的不频繁词元将被过滤掉。


In [ ]:
# 把句子拆分成词元列表
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # paragraphs[行，行中的句子]:List[List[Str]]
        # paragraph[句子列表]:List[Str]
        # d2l.tokenize:把句子拆分成词元列表:List[List[Str]]
        # 左边paragraphs[词元列表]:List[List[List[Str]]]
        # paragraphs: List[ List[ List[str] ] ]
        #             ↑      ↑      ↑
        #            段落   句子   词（token）
        paragraphs = [d2l.tokenize(paragraph, token='word') for paragraph in paragraphs]
        # [表达式 for 外层变量 in 外层可迭代对象 for 内层变量 in 内层可迭代对象]
        # paragraphs:List[List[List[Str]]]
        # paragraph:List[List[Str]]
        # sentence:List[Str]
        # sentences:List[List[Str]]:整个数据集的所有句子
        sentences = [sentence for paragraph in paragraphs for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=['<pad>', '<mask>', '<cls>', '<sep>'])
        
        examples = []
        # 分解
        # paragraphs:List[List[List[Str]]]
        # paragraph:List[List[Str]]
        # 获取下一句子预测任务NSP的数据
        # _get_nsp_data_from_paragraph:List[Tuple[List[str], List[int], bool]]
        # 分解
        # example:(tokens, segments, is_next)
        # 合并
        # examples:List[Tuple[List[str], List[int], bool]]
        # 此时examples列表中每个元素是一个元组(tokens, segments, is_next), 这个元组信息来源于之前的一个段落(1行)
        examples = [example for paragraph in paragraphs for example in _get_nsp_data_from_paragraph(paragraph, paragraphs, self.vocab, max_len)]
        for example in examples[:3]:
            print(example)
        # 获取遮蔽语言模型任务MLM的数据
        # 这里用到元组拼接，结果：(mlm_input_ids, pred_positions, mlm_pred_labels_ids, segments, is_next)
        # 在_get_mlm_data_from_tokens中，所有的词元都被转换为了词元ID
        examples = [_get_mlm_data_from_tokens(tokens, self.vocab) + (segments, is_next) for tokens, segments, is_next in examples]
        for example in examples[:3]:
            print(example)
        # 填充输入
        # _pad_bert_inputs:填充输入，返回元组
        # (self.all_token_ids, self.all_segments, self.valid_lens, self.all_pred_positions, self.all_mlm_weights, self.all_mlm_labels_ids, self.nsp_labels)
        (self.all_token_ids, self.all_segments, self.valid_lens, self.all_pred_positions, self.all_mlm_weights, self.all_mlm_labels_ids, self.nsp_labels) = _pad_bert_inputs(examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx],
                self.all_segments[idx],
                self.valid_lens[idx], 
                self.all_pred_positions[idx],
                self.all_mlm_weights[idx], 
                self.all_mlm_labels_ids[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

通过使用`_read_wiki`函数和`_WikiTextDataset`类，我们定义了下面的`load_data_wiki`来下载并生成WikiText-2数据集，并从中生成预训练样本。


In [125]:
#@save
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers()
    # paragraphs:List[List[Str]]
    paragraphs = _read_wiki()
    # train_set: all_token_ids, all_segments, valid_lens, all_pred_positions, all_mlm_weights, all_mlm_labels, nsp_labels
    train_set = _WikiTextDataset(paragraphs, max_len)
    # train_iter: all_token_ids, all_segments, valid_lens, all_pred_positions, all_mlm_weights, all_mlm_labels, nsp_labels
    train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

将批量大小设置为512，将BERT输入序列的最大长度设置为64，我们打印出小批量的BERT预训练样本的形状。注意，在每个BERT输入序列中，为遮蔽语言模型任务预测$10$（$64 \times 0.15$）个位置。


In [142]:
# max_len 表示每个样本的最大长度
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

(['<cls>', 'the', 'secwepemc', 'people', 'have', 'lived', 'in', 'the', 'adams', 'river', 'valley', 'for', 'millennia', '<sep>', 'a', '1977', 'study', 'by', 'the', 'provincial', 'government', 'along', 'the', 'lower', 'river', 'found', 'sixty', '@-@', 'six', 'sites', 'with', 'evidence', 'of', 'habitation', 'dating', 'to', '2000', 'bce', '<sep>'], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], True)
(['<cls>', 'the', 'soldiers', 'were', 'told', 'by', 'this', 'agent', 'that', '"', 'the', 'orders', 'were', 'to', 'kill', 'them', 'all', '"', ',', 'never', 'mind', 'that', 'only', 'civilians', ',', 'women', 'and', 'children', 'included', ',', 'were', 'found', '<sep>', 'the', 'anthem', 'was', 'first', 'performed', 'in', 'lithuania', 'during', 'the', 'great', '<unk>', 'of', '<unk>', 'on', 'december', '3', ',', '1905', '.', '<sep>'], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [104]:

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X, mlm_Y, nsp_y) in train_iter:
    print(f"{tokens_X.shape}\n"
          f"{segments_X.shape}\n"
          f"{valid_lens_x.shape}\n"
          f"{pred_positions_X.shape}\n"
          f"{mlm_weights_X.shape}\n"
          f"{mlm_Y.shape}\n"
          f"{nsp_y.shape}")
    break

torch.Size([512, 64])
torch.Size([512, 64])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512, 10])
torch.Size([512, 10])
torch.Size([512])


最后，我们来看一下词量。即使在过滤掉不频繁的词元之后，它仍然比PTB数据集的大两倍以上。


In [120]:
len(vocab)

20256

## 小结

* 与PTB数据集相比，WikiText-2数据集保留了原来的标点符号、大小写和数字，并且比PTB数据集大了两倍多。
* 我们可以任意访问从WikiText-2语料库中的一对句子生成的预训练（遮蔽语言模型和下一句预测）样本。

## 练习

1. 为简单起见，句号用作拆分句子的唯一分隔符。尝试其他的句子拆分技术，比如Spacy和NLTK。以NLTK为例，需要先安装NLTK：`pip install nltk`。在代码中先`import nltk`。然后下载Punkt语句词元分析器：`nltk.download('punkt')`。要拆分句子，比如`sentences = 'This is great ! Why not ?'`，调用`nltk.tokenize.sent_tokenize(sentences)`将返回两个句子字符串的列表：`['This is great !', 'Why not ?']`。
1. 如果我们不过滤出一些不常见的词元，词量会有多大？


[Discussions](https://discuss.d2l.ai/t/5738)
